In [1]:
import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier, Pool, cv

In [2]:
train_df = pd.read_csv('dataset.csv')

In [3]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,796,797,798,799,800,801,802,803,804,target
0,8.716706,1.913824,5.496236,-585.957061,519.236575,0.955239,1.007745,4.646661,1.420593,0.885670,...,8.100115,2.984066,14.273366,2.030239,65.628030,8.562931,7.661086,-501.658049,96.421634,0
1,705.270328,1.084182,706.415733,-1659.432876,-1364.472109,0.087307,0.080937,0.635242,701.729778,-0.062182,...,709.597244,-0.052859,15.762382,0.022418,67.183038,705.140579,702.406840,646.006837,98.005781,0
2,8.171244,0.996805,9.454911,-576.997401,-1133.758886,3.078693,2.984353,-1717.003823,0.064493,1.950616,...,4.534588,0.951612,15.018689,2.000632,66.404812,6.387988,4.552457,-124.111920,97.254314,0
3,8.926243,-0.097978,0.153883,1161.143765,-1936.510286,0.952026,3.187874,8.425161,1.411188,0.141314,...,0.452141,-0.177172,13.738149,0.062407,65.544608,7.695578,8.822014,-917.495768,96.282435,0
4,2.424358,2.088771,2.460280,1405.318204,-954.195401,0.975396,2.049270,8.722093,1.155668,3.106870,...,1.100180,1.102594,15.882553,-0.009456,67.661789,5.695057,2.206162,-1312.685923,98.284961,0


In [4]:
train_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,796,797,798,799,800,801,802,803,804,target
count,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,...,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000
mean,4.905736,1.004077,4.905259,-16.764958,11.378823,0.985953,0.991219,4.843143,4.818068,1.019979,...,4.971858,1.021051,15.134639,1.001243,66.538199,4.898069,4.927336,25.268862,97.299306,0.168437
std,572.099724,1.006358,572.239569,1144.194017,1166.478566,0.998380,1.021583,586.901682,572.144211,1.021059,...,572.067999,1.006967,0.720456,1.011120,0.722748,572.089702,572.102289,1138.873192,0.721738,0.374313
min,-1985.373872,-0.365879,-1979.222507,-1997.880619,-1999.071536,-0.371556,-0.326992,-1985.977156,-1978.848358,-0.359753,...,-1979.513449,-0.342419,13.738149,-0.267957,65.139060,-1979.963658,-1985.035268,-1998.404272,95.956766,0.000000
25%,1.733205,0.048172,1.689857,-1005.597646,-1005.410989,0.041556,0.044230,1.769651,1.632500,0.053863,...,1.736377,0.052952,14.417427,0.043592,65.809713,1.692510,1.689074,-936.466096,96.601876,0.000000
50%,5.066410,0.966576,5.005792,-33.068556,12.529704,0.961443,0.960883,5.196335,5.050662,0.969711,...,5.113488,0.968487,15.183182,0.968037,66.598725,5.003343,4.963494,22.042498,97.355167,0.000000
75%,8.302775,1.853334,8.283948,986.587994,1039.632650,1.848203,1.857232,8.377849,8.176830,1.865839,...,8.302549,1.882305,15.840340,1.833787,67.234482,8.332029,8.158775,1018.741960,98.005873,0.000000
max,1999.164279,6.018990,1998.083977,1999.948257,1998.248312,5.108544,6.127216,2006.079911,1996.010797,7.010780,...,1997.981103,6.964271,16.347344,5.969353,67.753568,1993.521727,2002.557362,1996.266269,98.530184,1.000000


In [5]:
X = train_df.drop('target', axis=1)
y = train_df['target']

In [6]:
X

,0,1,2,3,4,5,6,7,8,9,...,795,796,797,798,799,800,801,802,803,804
0,8.716706,1.913824,5.496236,-585.957061,519.236575,0.955239,1.007745,4.646661,1.420593,0.885670,...,-350.589112,8.100115,2.984066,14.273366,2.030239,65.628030,8.562931,7.661086,-501.658049,96.421634
1,705.270328,1.084182,706.415733,-1659.432876,-1364.472109,0.087307,0.080937,0.635242,701.729778,-0.062182,...,-310.431001,709.597244,-0.052859,15.762382,0.022418,67.183038,705.140579,702.406840,646.006837,98.005781
2,8.171244,0.996805,9.454911,-576.997401,-1133.758886,3.078693,2.984353,-1717.003823,0.064493,1.950616,...,963.773722,4.534588,0.951612,15.018689,2.000632,66.404812,6.387988,4.552457,-124.111920,97.254314
3,8.926243,-0.097978,0.153883,1161.143765,-1936.510286,0.952026,3.187874,8.425161,1.411188,0.141314,...,924.197433,0.452141,-0.177172,13.738149,0.062407,65.544608,7.695578,8.822014,-917.495768,96.282435
4,2.424358,2.088771,2.460280,1405.318204,-954.195401,0.975396,2.049270,8.722093,1.155668,3.106870,...,1538.930099,1.100180,1.102594,15.882553,-0.009456,67.661789,5.695057,2.206162,-1312.685923,98.284961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,-677.653589,0.994444,-669.003879,-1759.417087,-1817.409871,0.109886,-0.057440,7.439956,-673.217667,-0.280144,...,366.830377,-669.008008,1.020229,16.034417,0.117780,67.342090,-674.785620,-668.062000,965.488573,98.245640
3196,2.466394,0.884536,2.193199,558.080437,603.783366,0.873322,-0.078217,3.720081,0.345271,0.895536,...,-233.571831,10.101326,2.053870,16.143890,2.980016,67.361492,7.597098,0.238947,-1061.117383,98.206558
3197,9.059550,1.003965,4.557730,1459.647211,-1262.646830,2.005324,1.004275,1517.734031,0.563807,2.846424,...,-1938.052958,6.811463,-0.175868,14.130767,0.986677,65.680316,5.032263,7.984761,693.056872,96.179216
3198,8.777812,0.029054,6.866964,1019.305005,816.328748,-0.091580,2.116376,4.634465,4.228720,2.929335,...,1276.585511,2.539088,0.050971,16.080171,-0.072125,67.584982,0.547679,1.125976,755.648620,98.185440


In [7]:
y.describe()

count    3200.000000
mean        0.168437
std         0.374313
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: target, dtype: float64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [287]:
X_train_with_val, X_val, y_train_with_val, y_val = train_test_split(X_train, y_train, test_size=0.1,
                                                                    random_state=42, stratify=y_train)

In [378]:
from imblearn.over_sampling import SVMSMOTE

scaler = StandardScaler()
pca = PCA(n_components=3, random_state=42)
oversampler = SVMSMOTE(k_neighbors=10)

X_train_pca = pca.fit_transform(scaler.fit_transform(X_train))
X_train_sm, y_train_sm = oversampler.fit_resample(X_train_pca, y_train)
X_train_with_val_sm, X_val_sm, y_train_with_val_sm, y_val_sm = \
    train_test_split(X_train_sm, y_train_sm, test_size=0.1, random_state=42) #, stratify=y_train_sm)

In [373]:
model = CatBoostClassifier(eval_metric='AUC:hints=skip_train~false', logging_level='Silent',
                           early_stopping_rounds=150, random_state=42, auto_class_weights='Balanced')
model.fit(X_train_with_val_sm, y_train_with_val_sm, eval_set=(X_val_sm, y_val_sm))

In [376]:
# grid = {'learning_rate': np.logspace(-4, 0, 5),
#         'depth': np.arange(1, 20, 2),
#         'iterations': [10, 100, 200, 500, 750, 1000],
#         'border_count': [32, 64, 128, 200, 254],
#         'l2_leaf_reg': np.arange(1, 20, 2),
#        }

# grid_search_result = model.grid_search(grid, 
#                                        X=X_train_pca, 
#                                        y=y_train, 
#                                        plot=True,
#                                        verbose=False)

In [377]:
roc_auc_score(y_test, model.predict(pca.transform(scaler.transform(X_test))))

0.644597605123921

In [ ]:
# model = LogisticRegression()
# model.fit(X_train_pca, y_train)

In [393]:
from imblearn.over_sampling import RandomOverSampler

for _ in range(10):
    scaler = StandardScaler()
    pca = PCA(n_components=3, random_state=42)
    oversampler = RandomOverSampler()

    X_train_pca = pca.fit_transform(scaler.fit_transform(X_train))
    X_train_sm, y_train_sm = oversampler.fit_resample(X_train_pca, y_train)
    X_train_with_val_sm, X_val_sm, y_train_with_val_sm, y_val_sm = \
        train_test_split(X_train_sm, y_train_sm, test_size=0.1, random_state=42) #, stratify=y_train_sm)

    model = CatBoostClassifier(eval_metric='AUC:hints=skip_train~false', logging_level='Silent',
                               early_stopping_rounds=150, random_state=42) #, auto_class_weights='Balanced')
    model.fit(X_train_with_val_sm, y_train_with_val_sm, eval_set=(X_val_sm, y_val_sm))

    print(roc_auc_score(y_test, model.predict(pca.transform(scaler.transform(X_test)))))

0.7025898078529658
0.6784321915900864
0.663394597605124
0.6801726538568645
0.6781537176274018
0.6745335561125035
0.6560150375939849
0.6411166805903648
0.6652742968532441
0.6691729323308271


In [632]:
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.over_sampling import SMOTE

from sklearn.decomposition import FactorAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import BaggingClassifier

ans = 0
N = 10

for _ in range(N):
    avg_score = 0
    num_splits = 10

    for (train_index, test_index) in StratifiedKFold(num_splits).split(X, y):
        scaler = StandardScaler()
        fa = FactorAnalysis(n_components=3, max_iter=1000)
#         oversampler = RandomOverSampler()

        X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]
        X_train_fa = fa.fit_transform(scaler.fit_transform(X_train))

        estimator = DecisionTreeClassifier(criterion='log_loss')
        model = BalancedBaggingClassifier(estimator, n_estimators=500, oob_score=False)
        model.fit(X_train_fa, y_train)

        avg_score += roc_auc_score(y_test, model.predict(pca.transform(scaler.transform(X_test))))

    ans += avg_score / num_splits
    print(avg_score / num_splits)

print(f'Total Average: {ans / N}')

0.6978939144672271
0.7076340091743504
0.7052413764725336
0.7092104658036394
0.7095304747157509
0.7067213029393496
0.7141782786530786
0.70674513587103
0.7080769610644755
0.7046229703413127
Total Average: 0.7069854889502748


In [636]:
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import NearMiss
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import RUSBoostClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import FactorAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7, stratify=y)

scaler = StandardScaler()
fa = FactorAnalysis(n_components=3, random_state=42)

logloss_estimator = DecisionTreeClassifier(criterion='log_loss', random_state=42)
entropy_estimator = DecisionTreeClassifier(criterion='entropy', random_state=42)
gini_estimator = DecisionTreeClassifier(criterion='gini', random_state=42)

random_sampler = RandomUnderSampler(random_state=42)
cluster_sampler = ClusterCentroids(random_state=42)
near1_sampler = NearMiss(version=1)
near2_sampler = NearMiss(version=2)
near3_sampler = NearMiss(version=3)

model = BalancedBaggingClassifier(estimator, oob_score=True, random_state=42)

pipeline = Pipeline(steps=[('scaler', scaler), ('fa', fa), ('model', model)])

params = {
    'fa__max_iter': [100, 300, 500],
    'model__estimator': [logloss_estimator, entropy_estimator, gini_estimator],
    'model__n_estimators': [100, 300, 500],
    'model__oob_score': [True, False],
    'model__replacement': [True, False],
    'model__sampler': [random_sampler, cluster_sampler, near1_sampler, near2_sampler, near3_sampler]
}

search = GridSearchCV(pipeline, params, n_jobs=-1)
search.fit(X_train, y_train)

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('fa',
                                        FactorAnalysis(n_components=3,
                                                       random_state=42)),
                                       ('model',
                                        BalancedBaggingClassifier(estimator=DecisionTreeClassifier(),
                                                                  oob_score=True,
                                                                  random_state=42))]),
             n_jobs=-1,
             param_grid={'fa__max_iter': [100, 300, 500],
                         'model__estimator': [DecisionTreeClassifier(criterion='log_loss',
                                                                     random_state=42),
                                              DecisionTreeClassifier(criterion='entropy',
                                                                     random_state=42),
                                              DecisionTreeClassifier(random_state=42)],
                         'model__n_estimators': [100, 300, 500],
                         'model__oob_score': [True, False],
                         'model__replacement': [True, False],
                         'model__sampler': [RandomUnderSampler(random_state=42),
                                            ClusterCentroids(random_state=42),
                                            NearMiss(), NearMiss(version=2),
                                            NearMiss(version=3)]})

In [637]:
search.best_params_

{'fa__max_iter': 100,
 'model__estimator': DecisionTreeClassifier(criterion='log_loss', random_state=42),
 'model__n_estimators': 500,
 'model__oob_score': True,
 'model__replacement': True,
 'model__sampler': RandomUnderSampler(random_state=42)}

In [638]:
roc_auc_score(y_test, search.predict(X_test))

0.7059314954051796

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(
/Users/sergeyrid/opt/anaconda3/lib/python3.8/site-packages/imble

In [558]:
def avg_predict(X, models, t):
    return (sum([model.predict_proba(X) for model in models]) / len(models) < t)[:,0]

In [566]:
scaler = StandardScaler()
fa = FactorAnalysis(n_components=4, max_iter=10000, random_state=41)

X_train_fa = fa.fit_transform(scaler.fit_transform(X_train))

X_train_with_val, X_val, y_train_with_val, y_val = \
    train_test_split(X_train_fa, y_train, test_size=0.1, random_state=41, stratify=y_train)

for _ in range(10):
    model = BalancedBaggingClassifier(None, n_estimators=1000, oob_score=True)
    model.fit(X_train_fa, y_train)
    cat_model = CatBoostClassifier(eval_metric='AUC:hints=skip_train~false', logging_level='Silent',
                           early_stopping_rounds=150, auto_class_weights='Balanced')
    cat_model.fit(X_train_with_val, y_train_with_val, eval_set=(X_val, y_val))

    best_t = 0
    best_score = 0
    for t in np.arange(0.01, 1, 0.01):
        score = roc_auc_score(y_val, avg_predict(X_val, [model, cat_model], t))
        if score > best_score:
            best_t = t
            best_score = score
    print(best_t)

    print(roc_auc_score(y_test, avg_predict(pca.transform(scaler.transform(X_test)), [model, cat_model], best_t)))

0.52
0.6871345029239766
0.49
0.6780144806460595
0.5
0.6761347813979393


KeyboardInterrupt: 

In [ ]:
# Default score (all zeros): 0.5
# pca with 2 components: 0.619
# pca with 2 components and balanced class weights: 0.650
# pca with 3 comps: 0.669
# pca with 3 comps and stratify: 0.659
# with standardscaler: 0.692
# with gridsearch: 0.688

# EasyEnsembleClassifier: 0.695-0.715 (100 estimators)
# EasyEnsembleClassifier: 0.693-0.701 (500 estimators)
# EasyEnsembleClassifier with Catboost as estimator: 0.654

# Ensemble with FastICA: 0.682-0.719
# Ensemble with KernelPCA: 0.682-0.702
# Ensemble with IncrementalPCA: 0.684-0.701
# Ensemble with FactorAnalysis: 0.689-0.706
# Ensemble with FactorAnalysis (10000 iters): 0.691-0.712
# BalancedRandomForest with FactorAnalysis (4 comps, 10000 iters, 150 trees): 0.704-0.734
# BalancedBaggingClassifier with FactorAnalysis (4 comps, 10000 iters, 150 trees): 0.705-0.774
# BalancedBaggingClassifier with FactorAnalysis (4 comps, 10000 iters, 1000 trees): 0.722-0.746

# log_loss (max_iter = 1000, n_ests = 500, oob = False): 0.7523670286828181
# entropy (max_iter = 500): 0.7393483709273181
# gini (oob = False): 0.7208298524087997

# log_loss: 0.7083970158772666
# entropy: 0.7069854889502748
# gini: 0.7041350109898921

# Bagging + Factor, 4 comps, 10000 iters, 1000 estims, oob_score, 10-fold crossval: 0.695-0.703